# Remak

x,y座標を再度タグ付けできます。
road_following_A/apex/のデータを読み込み、road_following_REMAK/apex/に保存されます。

In [1]:
from jetracer.utils import preprocess
from jetracer.xy_dataset import XYDataset


KeyboardInterrupt: 

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import re
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox
from jupyter_clickable_image_widget import ClickableImageWidget
from jetcam.utils import bgr8_to_jpeg
import cv2
import torchvision.transforms as transforms
#from jetracer.utils import preprocess
#from jetracer.xy_dataset import XYDataset

DIR = "/home/jetson/jetracer/notebooks/"
TASK = 'road_following'
IMG_WIDTH = 224
IMG_HEIGHT = 224
CATEGORIES = ['apex']
DATASETS = ['A', 'B', 'REMARK']
LOAD_DATASETS = ['A', 'B']
SAVE_DATASETS = ['REMARK']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)
dataset = datasets[DATASETS[2]]

In [ ]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
picture_widget = ClickableImageWidget(width=224, height=224)
no_widget = ipywidgets.IntText(description='no')

In [ ]:
def load_img(no):
    global category_widget,load_datasets_widget,img,load_flag
    category_value = category_widget.value
    datasets_value = load_datasets_widget.value
    path = DIR + TASK + "_" + datasets_value + "/" + category_value + "/"
    filenames = os.listdir(path)
    if no > len(filenames):
        write_log("ファイルが存在しません。" + str(len(filenames)-1) + "以内の値を設定してください。")
        
    name = filenames[no]
        
    pattern = '(\d+)_(\d+).*'
    result = re.match(pattern, name)
    if result:
        x = result.group(1)
        y = result.group(2)
    img = cv2.imread(path + "/" + name)
    marked_img = img.copy()
    marked_img = cv2.circle(marked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
    picture_widget.value = bgr8_to_jpeg(marked_img)
    write_log(str(no) + "枚目の" + name + "を読込ました。")

def load_pic(c):
    global img,load_flag
    load_flag = True
    no = no_widget.value
    load_img(no)

def next_pic(c):
    global x,y,load_flag
    load_flag = True
    no = no_widget.value
    no = int(no) + 1
    no_widget.value = no
    load_img(no)
    
def save_snapshot(_, content, msg):
    global img,x,y,load_flag
    if content['event'] == 'click' and load_flag == True:
        load_flag = False
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        remarked_img = img.copy()
        remarked_img = cv2.circle(remarked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
        picture_widget.value = bgr8_to_jpeg(remarked_img)
        dataset.save_entry(CATEGORIES[0], img, x, y)
        write_log("新しい座標で保存しました。")
        

picture_widget.on_msg(save_snapshot)
    
load_pic_button = ipywidgets.Button(description='load')
next_pic_button = ipywidgets.Button(description='next')

load_pic_button.on_click(load_pic)
next_pic_button.on_click(next_pic)

load_datasets_widget = ipywidgets.Dropdown(options=LOAD_DATASETS, description='load dataset', index=0)
save_datasets_widget = ipywidgets.Dropdown(options=SAVE_DATASETS, description='save dataset')

category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='load category')

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([category_widget, load_datasets_widget]),
    ipywidgets.HBox([category_widget, save_datasets_widget]),
    ipywidgets.HBox([no_widget,load_pic_button,next_pic_button]),
    picture_widget,
    process_widget,
])
display(data_collection_widget)